## Obtain Data

In this notebook we perform the following steps:
* Establish the first hour of the dataset
* For the first month,
  * Obtain a list of available stations by state
  * Obtain temperature observations from weather stations in the MISO footprint
    * Stations are organized into MISO regions by state boundaries
    * Stations are predominantly clustered in population centers, making many observation redundant
    * There are many lacunae in some stations
  * Obtain the actual hourly MISO Load data and historical Medium-Term Load Forecasts (MTLF)
  * Join Load and MTLF data with weather observations to complete the raw data
  * Persist the data and demonstrate use of dataset wrapper class
* Update the dataset to the current day
* Identify and mitigate lacunae
* Publish the data

### Define Date Ranges

In [1]:
%load_ext autoreload
%autoreload 2

from MISO import prevailing_time as est

# beginning of MISO's historical records that include the southern region (zones 8-10)
first_hour = est(2015, 2, 1, 0)
# five hours weather data was lost for 2015-06-17
# first_hour = est(2015, 7, 1, 0)

# latest date with actual load data available is
# l = date.today() - timedelta(days=2)
# last_hour = est(l.year, l.month, l.day, 23)
# instead, fix the date for repeatbility
last_hour = est(2022, 3, 31, 23)



### Obtain Weather Data for Zones

Source: [Iowa State ASOS Network Downloads](https://mesonet.agron.iastate.edu/request/download.phtml)

`zones` below represents an initial candidate set of weather stations in each
zone. We will examine the data quality from each of the stations in the next
section.

In [2]:
from weather_data import ASOS
from pathlib import Path
from os.path import isfile
import pandas as pd

zones = { 1 : {'MN': ['MSP',  # Minneapolis / St. Paul (STP)
                      'RST',  # Rochester
                      'DYT'], # Duluth
               'ND': ['FAR',  # Fargo
                      'BIS',  # Bismarck
                      'GFK'], # Grand Forks
               'SD': ['ABR'], # Aberdeen
               'WI': ['LSE'], # La Crosse
               'IL': ['SFY']  # Savanna 
              },
          2 : {'WI': ['MSN', 'MKE', 'EAU', 'GRB'],
               'MI': ['ANJ', 'SAW', 'IWD']},
          3 : {'IA': ['DSM', 'CID', 'DVN', 'SUX', 'ALO', 'MCW']}
        }

def download_file_path(zone, state, station):
    zone_data = f"./data/zone_{zone}"
    Path(zone_data).mkdir(exist_ok=True)
    return f'{zone_data}/{state}_{station}.parquet'

asos = ASOS()
def download_station(zone, state, station):
    path = download_file_path(zone, state, station) 
    if isfile(path):
        return pd.read_parquet(path)

    station = asos.get_hourly_observations(station, first_hour, last_hour)
    if station is None:
        print(f'Retrieve {station} failed')
        return None
    station.to_parquet(path)
    return station

from multiprocessing import cpu_count
from joblib import Parallel, delayed
def do_parallel(func, zones):
    parallel = Parallel(n_jobs=cpu_count())
    result = {}
    for zone in zones:
        stations = [(state, station) for state in zones[zone].keys() for station in zones[zone][state]]
        result[zone] = pd.concat(parallel(delayed(func)(zone, state, station) for (state, station) in stations))
    return result

In [3]:
zone_results = do_parallel(download_station, zones)
all_zones = pd.concat(zone_results.values())

### Weather Data Quality

We will look for large lacunae in the original data by examining where a few hours or more were interpolated.

In [4]:
interpolated = all_zones[pd.isna(all_zones['observation_time'])].copy()
f = interpolated.groupby([interpolated.index.date, 'station']).temp.count()
f[f > 4].groupby(level='station').count()

station
ABR     20
ALO     10
ANJ     58
BIS     12
CID     30
DSM      1
DVN     32
DYT     66
EAU     15
FAR     10
GFK     20
GRB     11
IWD    126
LSE     34
MCW     51
MKE      6
MSN     11
MSP      4
RST     12
SAW     11
SFY    123
SUX     50
Name: temp, dtype: int64

In [5]:
f[f > 4].groupby(level='station').max()

station
ABR    18
ALO    23
ANJ    24
BIS    11
CID    23
DSM     6
DVN    24
DYT    24
EAU    24
FAR    16
GFK    11
GRB    11
IWD    24
LSE    24
MCW    24
MKE     9
MSN     8
MSP    11
RST    21
SAW     8
SFY    24
SUX    24
Name: temp, dtype: int64

### Best Stations for Each Zone

Based on the above results, we select stations in each zone with the fewest lacunae in the dataset.

In [6]:
zone_stations = {1: 'MSP', 2: 'MKE', 3: 'DSM' }

#### Feature Engineering: Temperature Differences

In [4]:
df = asos.get_hourly_observations('MSP', first_hour, last_hour)
df1 = df.pivot(columns='station', values='temp')
df2 = df1.shift(periods=1)
df2.iloc[0] = df2.iloc[1] #replace NaN
df2.head()

Fetching https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=tmpf&data=feel&tz=Etc/UTC&format=comma&latlon=yes&year1=2015&month1=1&day1=25&year2=2022&month2=4&day2=9&station=MSP


station,MSP
hour,
2015-02-01 00:00:00-05:00,23.00
2015-02-01 01:00:00-05:00,23.00
2015-02-01 02:00:00-05:00,21.02
2015-02-01 03:00:00-05:00,19.04
2015-02-01 04:00:00-05:00,19.04


In [9]:
df1.head()

station,MSP
hour,
2015-02-01 00:00:00-05:00,23.00
2015-02-01 01:00:00-05:00,21.02
2015-02-01 02:00:00-05:00,19.04
2015-02-01 03:00:00-05:00,19.04
2015-02-01 04:00:00-05:00,17.06


In [5]:
df_rst = asos.get_hourly_observations('RST', first_hour, last_hour)

Fetching https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=tmpf&data=feel&tz=Etc/UTC&format=comma&latlon=yes&year1=2015&month1=1&day1=25&year2=2022&month2=4&day2=9&station=RST


In [11]:
newdf = df1.join(df_rst.pivot(columns='station', values='temp'))

### Obtain the Regional MTLF and Actual Load for each Observation Hour

In [12]:
from MISO import MarketReports

miso = MarketReports('./data/mtlf')
actuals = miso.regional_hourly_load(first_hour, last_hour)
actuals.head()

,Market Day,HourEnding,MISO MTLF (MWh),MISO ActualLoad (MWh),North MTLF (MWh),North ActualLoad (MWh),South MTLF (MWh),South ActualLoad (MWh),Central MTLF (MWh),Central ActualLoad (MWh)
market_hour,,,,,,,,,,
2015-02-01 00:00:00-05:00,2015-02-01,1,69748,68899.08,15885,16150.30,16067,16163.39,37796,36585.39
2015-02-01 01:00:00-05:00,2015-02-01,2,67697,66793.78,15517,15713.40,15591,15571.34,36589,35509.04
2015-02-01 02:00:00-05:00,2015-02-01,3,66526,65489.85,15187,15381.56,15272,15137.39,36067,34970.90
2015-02-01 03:00:00-05:00,2015-02-01,4,66037,64884.03,15060,15239.59,15122,14935.61,35855,34708.83
2015-02-01 04:00:00-05:00,2015-02-01,5,65942,64689.33,15027,15229.09,15080,14750.43,35835,34709.81


In [13]:
from MISO import MarketReports

zonal_actuals = miso.zonal_hourly_load(first_hour, last_hour)
zonal_actuals.head()

,Market Day,HourEnding,MISO MTLF (MWh),MISO ActualLoad (MWh),LRZ1 MTLF (MWh),LRZ1 ActualLoad (MWh),LRZ2_7 MTLF (MWh),LRZ2_7 ActualLoad (MWh),LRZ3_5 MTLF (MWh),LRZ3_5 ActualLoad (MWh),LRZ4 MTLF (MWh),LRZ4 ActualLoad (MWh),LRZ6 MTLF (MWh),LRZ6 ActualLoad (MWh),LRZ8_9_10 MTLF (MWh),LRZ8_9_10 ActualLoad (MWh)
market_hour,,,,,,,,,,,,,,,,
2015-02-01 00:00:00-05:00,2015-02-01,1,69748,68899.08,11099,11337.89,17257,16097.85,9479,9406.37,5183,5177.45,10663,10716.13,16067,16163.39
2015-02-01 01:00:00-05:00,2015-02-01,2,67697,66793.78,10829,11014.87,16376,15579.76,9280,9161.09,5061,5016.36,10560,10450.36,15591,15571.34
2015-02-01 02:00:00-05:00,2015-02-01,3,66526,65489.85,10565,10795.37,16077,15290.55,9148,8985.26,4980,4953.68,10484,10327.60,15272,15137.39
2015-02-01 03:00:00-05:00,2015-02-01,4,66037,64884.03,10468,10714.42,15942,15149.88,9087,8916.82,4940,4904.88,10478,10262.42,15122,14935.61
2015-02-01 04:00:00-05:00,2015-02-01,5,65942,64689.33,10432,10700.09,15865,15198.61,9085,8923.20,4936,4895.37,10544,10221.63,15080,14750.43


### Combine Features with Actuals

In [14]:
data = newdf.join(zonal_actuals, how='inner')

df.shape[0] == zonal_actuals.shape[0] == data.shape[0]

True

In [15]:
data.head()

,MSP,RST,Market Day,HourEnding,MISO MTLF (MWh),MISO ActualLoad (MWh),LRZ1 MTLF (MWh),LRZ1 ActualLoad (MWh),LRZ2_7 MTLF (MWh),LRZ2_7 ActualLoad (MWh),LRZ3_5 MTLF (MWh),LRZ3_5 ActualLoad (MWh),LRZ4 MTLF (MWh),LRZ4 ActualLoad (MWh),LRZ6 MTLF (MWh),LRZ6 ActualLoad (MWh),LRZ8_9_10 MTLF (MWh),LRZ8_9_10 ActualLoad (MWh)
hour,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00-05:00,23.00,24.08,2015-02-01,1,69748,68899.08,11099,11337.89,17257,16097.85,9479,9406.37,5183,5177.45,10663,10716.13,16067,16163.39
2015-02-01 01:00:00-05:00,21.02,24.08,2015-02-01,2,67697,66793.78,10829,11014.87,16376,15579.76,9280,9161.09,5061,5016.36,10560,10450.36,15591,15571.34
2015-02-01 02:00:00-05:00,19.04,23.00,2015-02-01,3,66526,65489.85,10565,10795.37,16077,15290.55,9148,8985.26,4980,4953.68,10484,10327.60,15272,15137.39
2015-02-01 03:00:00-05:00,19.04,19.94,2015-02-01,4,66037,64884.03,10468,10714.42,15942,15149.88,9087,8916.82,4940,4904.88,10478,10262.42,15122,14935.61
2015-02-01 04:00:00-05:00,17.06,19.04,2015-02-01,5,65942,64689.33,10432,10700.09,15865,15198.61,9085,8923.20,4936,4895.37,10544,10221.63,15080,14750.43


## Feature Engineering: Business Hours & Day of Year

In [12]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay, BusinessHour

federal_business_days = CustomBusinessDay(calendar=USFederalHolidayCalendar())
bh = BusinessHour()
def is_biz_hour(d):
    return federal_business_days.is_on_offset(d) and bh.is_on_offset(d)

data['IsBusinessHour'] = data.index.to_series().apply(lambda d: 1 if is_biz_hour(d) else 0)
data['DayOfYear'] = data['Market Day'].dt.day_of_year

In [19]:
zone1_cols = ['MSP', 'RST', 'LRZ1 MTLF (MWh)', 'LRZ1 ActualLoad (MWh)']
data[zone1_cols].head()

,MSP,RST,LRZ1 MTLF (MWh),LRZ1 ActualLoad (MWh)
hour,,,,
2015-02-01 00:00:00-05:00,23.00,24.08,11099,11337.89
2015-02-01 01:00:00-05:00,21.02,24.08,10829,11014.87
2015-02-01 02:00:00-05:00,19.04,23.00,10565,10795.37
2015-02-01 03:00:00-05:00,19.04,19.94,10468,10714.42
2015-02-01 04:00:00-05:00,17.06,19.04,10432,10700.09


In [18]:
data[zone1_cols].to_parquet(f'data/zone1_rst.parquet')